<hr style="border:2px solid gray">

##### Author  : SIVA VIKNESH 
##### Email   : siva.viknesh@sci.utah.edu / sivaviknesh14@gmail.com 
##### Address : SCI INSTITUTE, UNIVERSITY OF UTAH, SALT LAKE CITY, UTAH, USA 
<hr style="border:2px solid gray">

In [ ]:
import os
import torch
import math
import torch.nn as nn
import torch.optim as optim
from torch.nn.parameter import Parameter
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib.ticker as plticker
from mpl_toolkits.mplot3d import Axes3D
from scipy import integrate

rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = [12, 12]
import vtk
from vtk.util import numpy_support as VN
from itertools import combinations

Lorenz system of differential equations:

$$
\begin{aligned}
\dot{x} & = \sigma(y-x) \\
\dot{y} & = \rho x - y - xz \\
\dot{z} & = -\beta z + xy
\end{aligned}
$$

Parameters: \\(\sigma\\), \\(\beta\\), and \\(\rho\\)

In [ ]:
## Simulate the Lorenz System

dt = 0.01
T = 50
t = np.arange(0,T+dt,dt)
beta = 8/3
sigma = 10
rho = 28

# CASE I
def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
    x, y, z = x_y_z
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]
np.random.seed(123)
x0 = (-8, 7, 27)
x_t = integrate.odeint(lorenz_deriv, x0, t,rtol=10**(-12),atol=10**(-12)*np.ones_like(x0))
x, y, z = x_t.T

fig,ax = plt.subplots(1,1,subplot_kw={'projection': '3d'})
plt.plot(x, y, z, linewidth=1)
ax.view_init(30, 140)
plt.show()

In [ ]:
# COMPUTING THE COMBINATIONS AMONG THE THREE CHOSEN TEMPORAL MODES
def poolData(yin,nVars,polyorder):
    n = yin.shape[0]
    yout = np.zeros((n,1))

    # poly order 0
    yout[:,0] = np.ones(n)

    # poly order 1
    for i in range(nVars):
        yout = np.append(yout,yin[:,i].reshape((yin.shape[0],1)),axis=1)

    # poly order 2
    if polyorder >= 2:
        for i in range(nVars):
            for j in range(i,nVars):
                yout = np.append(yout,(yin[:,i]*yin[:,j]).reshape((yin.shape[0],1)),axis=1)

    # poly order 3
    if polyorder >= 3:
        for i in range(nVars):
            for j in range(i,nVars):
                for k in range(j,nVars):
                    yout = np.append(yout,(yin[:,i]*yin[:,j]*yin[:,k]).reshape((yin.shape[0],1)),axis=1)

    return yout

def DERIVATIVE(a, b, c):
    dadt1 = sigma*(b - a)
    dadt2 = rho*a - b - a*c
    dadt3 = -beta*c + a*b
    
    return torch.vstack((dadt1, dadt2, dadt3))

A_candidates = poolData (x_t, 3, 2)

In [ ]:
processor = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("AVAILABLE PROCESSOR:", processor, '\n')

N_modes  = 3
x        = torch.Tensor(x).to(processor)
y        = torch.Tensor(y).to(processor)
z        = torch.Tensor(z).to(processor)
t        = torch.Tensor(t).to(processor)

A_candidates  = torch.Tensor (A_candidates).to(processor)
A1A2A3_time_deriv = torch.transpose(DERIVATIVE (x, y, z), 0, 1)


# HYPERPARAMETERS FOR THE SINDy POD METHODOLOGY
Epochs        = 50000
learning_rate = 1e-1
step_epoch    = 3500
decay_rate    = 0.50

N_terms = math.comb(N_modes, 1) + math.comb(N_modes, 2) + math.comb(N_modes, 3) + N_modes
                       

fig, ax = plt.subplots(nrows = 3, ncols = 1, figsize =(20, 10))
fig.suptitle('Ground-truth data')

ax[0].plot(A1A2A3_time_deriv[:, 0].detach().cpu().numpy(), '-o') 
ax[0].set_title('$\dot{X}$')

ax[1].plot(A1A2A3_time_deriv[:, 1].detach().cpu().numpy(), '-o') 
ax[1].set_title('$\dot{Y}$')

ax[2].plot(A1A2A3_time_deriv[:, 2].detach().cpu().numpy(), '-o') 
ax[2].set_title('$\dot{Z}$')
fig.tight_layout()
plt.show()

In [ ]:
class SINE_TERM (nn.Module):
    def __init__(self, b):
        super().__init__()
        self.b = b
        
    def forward(self, x):
        output = torch.sin(self.b*x)
        return output
    
class COSINE_TERM (nn.Module):
    def __init__(self, b):
        super().__init__()
        self.b = b
        
    def forward(self, x):
        output = torch.cos(self.b*x)
        return output

In [ ]:
class ADAPTIVE_SINDy_MODEL(nn.Module):
    def __init__(self, a, asine, acosine):
        super().__init__()    
        self.a        = a
        self.asine    = asine
        self.acosine  = acosine
        self.sine     = SINE_TERM   (self.asine)
        self.cosine   = COSINE_TERM (self.acosine)
        
    def forward(self, x):
        output_sine    = self.sine   (x)
        output_cosine  = self.cosine (x)
        output = torch.hstack((x, output_sine, output_cosine)) @ self.a
        return output
    
class SINDy_MODEL(nn.Module):
    def __init__(self, a):
        super().__init__()    
        self.a        = a
        
    def forward(self, x):

        output = x @ self.a
        return output

In [ ]:
# AMPLITUDE COEFFICIENTS OF SINDy MODEL
COEFF_ADT1 = torch.ones(A_candidates.shape[1], N_modes, requires_grad= True, device= processor)
COEFF_ADT2 = torch.ones(A_candidates.shape[1], N_modes, requires_grad= True, device= processor)
COEFF_ADT3 = torch.ones(A_candidates.shape[1], N_modes, requires_grad= True, device= processor)
COEFF_ADT4 = torch.ones(A_candidates.shape[1], N_modes, requires_grad= True, device= processor)
COEFF_ADT5 = torch.ones(A_candidates.shape[1], N_modes, requires_grad= True, device= processor)
COEFF_ADT6 = torch.ones(A_candidates.shape[1], N_modes, requires_grad= True, device= processor)

In [ ]:
# COEFFICIENTS OF SINDy MODEL
optim_COEFF_ADT1 = optim.Adam([COEFF_ADT1], lr=learning_rate, betas = (0.9,0.99),eps = 10**-15)
optim_COEFF_ADT2 = optim.Adam([COEFF_ADT2], lr=learning_rate, betas = (0.9,0.99),eps = 10**-15)
optim_COEFF_ADT3 = optim.Adam([COEFF_ADT3], lr=learning_rate, betas = (0.9,0.99),eps = 10**-15)
optim_COEFF_ADT4 = optim.Adam([COEFF_ADT4], lr=learning_rate, betas = (0.9,0.99),eps = 10**-15)
optim_COEFF_ADT5 = optim.Adam([COEFF_ADT5], lr=learning_rate, betas = (0.9,0.99),eps = 10**-15)
optim_COEFF_ADT6 = optim.Adam([COEFF_ADT6], lr=learning_rate, betas = (0.9,0.99),eps = 10**-15)

# WEIGHT FUNCTION OF SINDy MODEL
WEIGHTS3     = Parameter(torch.ones_like(COEFF_ADT3), requires_grad= True)
WEIGHTS5     = Parameter(torch.ones_like(COEFF_ADT5), requires_grad= True)
WEIGHTS6     = Parameter(torch.ones_like(COEFF_ADT6), requires_grad= True)
Lambda1      = Parameter(torch.tensor(0.01),  requires_grad= True)
Lambda2      = Parameter(torch.tensor(0.015), requires_grad= True)
Lambda3      = Parameter(torch.tensor(0.01),  requires_grad= True)
Lambda4      = Parameter(torch.tensor(1.0),   requires_grad= True)
Lambda5      = Parameter(torch.tensor(1.0),   requires_grad= True)

optim_weights3 = optim.Adam([WEIGHTS3], lr = learning_rate, betas = (0.9,0.99),eps = 10**-15)
optim_weights5 = optim.Adam([WEIGHTS5], lr = learning_rate, betas = (0.9,0.99),eps = 10**-15)
optim_weights6 = optim.Adam([WEIGHTS6], lr = learning_rate, betas = (0.9,0.99),eps = 10**-15)

optim_Lambda4  = optim.Adam([Lambda4],  lr = learning_rate, betas = (0.9,0.99),eps = 10**-15)
optim_Lambda5  = optim.Adam([Lambda5],  lr = learning_rate, betas = (0.9,0.99),eps = 10**-15)

# STEP DECAY DYNAMIC LEARNING RATE
scheduler_ADT1   = torch.optim.lr_scheduler.StepLR(optim_COEFF_ADT1, step_size=step_epoch, gamma=decay_rate)
scheduler_ADT2   = torch.optim.lr_scheduler.StepLR(optim_COEFF_ADT2, step_size=step_epoch, gamma=decay_rate)
scheduler_ADT3   = torch.optim.lr_scheduler.StepLR(optim_COEFF_ADT3, step_size=step_epoch, gamma=decay_rate)
scheduler_ADT4   = torch.optim.lr_scheduler.StepLR(optim_COEFF_ADT4, step_size=step_epoch, gamma=decay_rate)
scheduler_ADT5   = torch.optim.lr_scheduler.StepLR(optim_COEFF_ADT5, step_size=step_epoch, gamma=decay_rate)
scheduler_ADT6   = torch.optim.lr_scheduler.StepLR(optim_COEFF_ADT6, step_size=step_epoch, gamma=decay_rate)

scheduler_weights3 = torch.optim.lr_scheduler.StepLR(optim_weights3, step_size=step_epoch, gamma=decay_rate)
scheduler_weights5 = torch.optim.lr_scheduler.StepLR(optim_weights5, step_size=step_epoch, gamma=decay_rate)
scheduler_weights6 = torch.optim.lr_scheduler.StepLR(optim_weights6, step_size=step_epoch, gamma=decay_rate)

scheduler_LAMBDA4  = torch.optim.lr_scheduler.StepLR(optim_Lambda4, step_size=step_epoch, gamma=decay_rate)
scheduler_LAMBDA5  = torch.optim.lr_scheduler.StepLR(optim_Lambda5, step_size=step_epoch, gamma=decay_rate)

### CASE 1: SINDy with fixed $\lambda$ parameter = 0.01

In [ ]:
# TEMPORAL MODE 1
A1_DT1 = SINDy_MODEL(COEFF_ADT1 [:, 0]).to(processor)

# TEMPORAL MODE 2
A2_DT1 = SINDy_MODEL(COEFF_ADT1 [:, 1]).to(processor)

# TEMPORAL MODE 3
A3_DT1 = SINDy_MODEL(COEFF_ADT1 [:, 2]).to(processor)

Loss_data1    = torch.empty(size=(Epochs, 1))
loss_function = nn.MSELoss()

for epoch in range(Epochs):
    A1_out, A2_out, A3_out  = A1_DT1 (A_candidates), A2_DT1 (A_candidates), A3_DT1 (A_candidates)
    output_data  = torch.stack((A1_out , A2_out, A3_out), dim = 1)
    loss_epoch   = loss_function (A1A2A3_time_deriv, output_data) + Lambda1*torch.linalg.matrix_norm(COEFF_ADT1, ord =1)
    
    optim_COEFF_ADT1.zero_grad()
    loss_epoch.backward()

    with torch.no_grad():
        optim_COEFF_ADT1.step()
        Loss_data1 [epoch] = loss_epoch.detach()
        COEFF_ADT1 [torch.abs(COEFF_ADT1) <= 0.005] = 0.0
        
    print('LOSS DATA, [EPOCH =', epoch,  ']:',  Loss_data1 [epoch].item())
    print('LEARNING RATE:', optim_COEFF_ADT1.param_groups[0]['lr'])
    print ("*"*85)
       
    scheduler_ADT1.step()

In [ ]:
threshold = 0.0005
#****************************************************************************#
loc = plticker.MultipleLocator(base=3) # this locator puts ticks at regular intervals
fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize =(14, 4))
fig.suptitle('$\lambda_{fixed} = 0.01$')

ax[0].plot(COEFF_ADT1 [:, 0].detach().cpu().numpy(), 'o') 
ax[0].set_title('X')
ax[0].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[0].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[0].xaxis.set_major_locator(loc)

ax[1].plot(COEFF_ADT1 [:, 1].detach().cpu().numpy(), 'o') 
ax[1].set_title('Y')
ax[1].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[1].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[1].xaxis.set_major_locator(loc)

ax[2].plot(COEFF_ADT1 [:, 2].detach().cpu().numpy(), 'o') 
ax[2].set_title('Z')
ax[2].axhline(y = threshold, color = 'r', linestyle = '-')
ax[2].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[2].xaxis.set_major_locator(loc)
fig.subplots_adjust(top=0.8)
fig.tight_layout()
plt.show()

print(COEFF_ADT1)


### CASE 2: SINDy with fixed $\lambda$ parameter = 0.015

In [ ]:
# TEMPORAL MODE 1
A1_DT2 = SINDy_MODEL(COEFF_ADT2 [:, 0]).to(processor)

# TEMPORAL MODE 2
A2_DT2 = SINDy_MODEL(COEFF_ADT2 [:, 1]).to(processor)

# TEMPORAL MODE 3
A3_DT2 = SINDy_MODEL(COEFF_ADT2 [:, 2]).to(processor)

Loss_data2    = torch.empty(size=(Epochs, 1))
loss_function = nn.MSELoss()

for epoch in range(Epochs):
    A1_out, A2_out, A3_out  = A1_DT2 (A_candidates), A2_DT2 (A_candidates), A3_DT2 (A_candidates)
    output_data  = torch.stack((A1_out , A2_out, A3_out), dim = 1)
    loss_epoch   = loss_function (A1A2A3_time_deriv, output_data) + Lambda2*torch.linalg.matrix_norm(COEFF_ADT2, ord =1)
    
    optim_COEFF_ADT2.zero_grad()
    loss_epoch.backward()

    with torch.no_grad():
        optim_COEFF_ADT2.step()
        Loss_data2 [epoch] = loss_epoch.detach()
        COEFF_ADT2 [torch.abs(COEFF_ADT2) < 0.005] = 0.0
        
    print('LOSS DATA, [EPOCH =', epoch,  ']:',  Loss_data2 [epoch].item())
    print('LEARNING RATE:', optim_COEFF_ADT2.param_groups[0]['lr'])
    print ("*"*85)
       
    scheduler_ADT2.step()

In [ ]:
plt.plot(Loss_data2.detach().cpu().numpy())
plt.xlabel('Epoch') 
plt.ylabel('Loss')
plt.yscale("log")
plt.show()


loc = plticker.MultipleLocator(base=3) # this locator puts ticks at regular intervals
fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize =(14, 4))
fig.suptitle('$\lambda_{fixed} = 0.015$')

ax[0].plot(COEFF_ADT2 [:, 0].detach().cpu().numpy(), 'o') 
ax[0].set_title('X')
ax[0].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[0].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[0].xaxis.set_major_locator(loc)

ax[1].plot(COEFF_ADT2 [:, 1].detach().cpu().numpy(), 'o') 
ax[1].set_title('Y')
ax[1].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[1].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[1].xaxis.set_major_locator(loc)

ax[2].plot(COEFF_ADT2 [:, 2].detach().cpu().numpy(), 'o') 
ax[2].set_title('Z')
ax[2].axhline(y = threshold, color = 'r', linestyle = '-')
ax[2].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[2].xaxis.set_major_locator(loc)
fig.subplots_adjust(top=0.8)
fig.tight_layout()
plt.show()

print(COEFF_ADT2)

### CASE 3: SINDy with fixed $\lambda$ parameter = 0.01 and adaptive weight

In [ ]:
# TEMPORAL MODE 1
A1_DT3 = SINDy_MODEL(COEFF_ADT3 [:, 0]).to(processor)

# TEMPORAL MODE 2
A2_DT3 = SINDy_MODEL(COEFF_ADT3 [:, 1]).to(processor)

# TEMPORAL MODE 3
A3_DT3 = SINDy_MODEL(COEFF_ADT3 [:, 2]).to(processor)

Loss_data3    = torch.empty(size=(Epochs, 1))
loss_function = nn.MSELoss()

for epoch in range(Epochs):
    A1_out, A2_out, A3_out  = A1_DT3 (A_candidates), A2_DT3 (A_candidates), A3_DT3 (A_candidates)
    output_data  = torch.stack((A1_out , A2_out, A3_out), dim = 1)
    loss_epoch   = loss_function (A1A2A3_time_deriv, output_data) + Lambda3*torch.linalg.matrix_norm(torch.abs(WEIGHTS3)*COEFF_ADT3, ord =1)
    
    optim_COEFF_ADT3.zero_grad()
    optim_weights3.zero_grad()
    loss_epoch.backward()

    with torch.no_grad():
        optim_COEFF_ADT3.step()
        optim_weights3.step()
        Loss_data3 [epoch] = loss_epoch.detach()
        COEFF_ADT3 [torch.abs(COEFF_ADT3) < 0.005] = 0.0
        
    print('LOSS DATA, [EPOCH =', epoch,  ']:',  Loss_data3 [epoch].item())
    print('LEARNING RATE:', optim_COEFF_ADT3.param_groups[0]['lr'])
    print ("*"*85)
       
    scheduler_ADT3.step()
    scheduler_weights3.step()

In [ ]:
plt.plot(Loss_data3.detach().cpu().numpy())
plt.xlabel('Epoch') 
plt.ylabel('Loss')
plt.yscale("log")
plt.show()


loc = plticker.MultipleLocator(base=3) # this locator puts ticks at regular intervals
fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize =(14, 4))
fig.suptitle('$\lambda_{fixed} = 0.01$ with Adaptive weights')

ax[0].plot(COEFF_ADT3 [:, 0].detach().cpu().numpy(), 'o') 
ax[0].set_title('X')
ax[0].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[0].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[0].xaxis.set_major_locator(loc)

ax[1].plot(COEFF_ADT3 [:, 1].detach().cpu().numpy(), 'o') 
ax[1].set_title('Y')
ax[1].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[1].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[1].xaxis.set_major_locator(loc)

ax[2].plot(COEFF_ADT3 [:, 2].detach().cpu().numpy(), 'o') 
ax[2].set_title('Z')
ax[2].axhline(y = threshold, color = 'r', linestyle = '-')
ax[2].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[2].xaxis.set_major_locator(loc)
fig.subplots_adjust(top=0.8)
fig.tight_layout()
plt.show()

print(COEFF_ADT3)

### CASE 4: SINDy with adaptive $\lambda$ parameter

In [ ]:
# TEMPORAL MODE 1
A1_DT4 = SINDy_MODEL(COEFF_ADT4 [:, 0]).to(processor)

# TEMPORAL MODE 2
A2_DT4 = SINDy_MODEL(COEFF_ADT4 [:, 1]).to(processor)

# TEMPORAL MODE 3
A3_DT4 = SINDy_MODEL(COEFF_ADT4 [:, 2]).to(processor)

Loss_data4    = torch.empty(size=(Epochs, 1))
loss_function = nn.MSELoss()

for epoch in range(Epochs):
    A1_out, A2_out, A3_out  = A1_DT4 (A_candidates), A2_DT4 (A_candidates), A3_DT4 (A_candidates)
    output_data  = torch.stack((A1_out , A2_out, A3_out), dim = 1)
    loss_epoch   = loss_function (A1A2A3_time_deriv, output_data) + torch.abs(Lambda4 + 1e-12)*torch.linalg.matrix_norm(COEFF_ADT4, ord =1)
    
    optim_COEFF_ADT4.zero_grad()
    optim_Lambda4.zero_grad()
    loss_epoch.backward()

    with torch.no_grad():
        optim_COEFF_ADT4.step()
        optim_Lambda4.step()
        Loss_data4 [epoch] = loss_epoch.detach()
        COEFF_ADT4 [torch.abs(COEFF_ADT4) < 0.005] = 0.0
        
    print('LOSS DATA, [EPOCH =', epoch,  ']:',  Loss_data4 [epoch].item())
    print('LEARNING RATE:', optim_COEFF_ADT4.param_groups[0]['lr'])
    print ("*"*85)
       
    scheduler_ADT4.step()
    scheduler_LAMBDA4.step()
    

In [ ]:
plt.plot(Loss_data4.detach().cpu().numpy())
plt.xlabel('Epoch') 
plt.ylabel('Loss')
plt.yscale("log")
plt.show()

loc = plticker.MultipleLocator(base=3) # this locator puts ticks at regular intervals
fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize =(14, 4))
fig.suptitle('Adaptive $\lambda $')

ax[0].plot(COEFF_ADT4 [:, 0].detach().cpu().numpy(), 'o') 
ax[0].set_title('X')
ax[0].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[0].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[0].xaxis.set_major_locator(loc)

ax[1].plot(COEFF_ADT4 [:, 1].detach().cpu().numpy(), 'o') 
ax[1].set_title('Y')
ax[1].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[1].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[1].xaxis.set_major_locator(loc)

ax[2].plot(COEFF_ADT4 [:, 2].detach().cpu().numpy(), 'o') 
ax[2].set_title('Z')
ax[2].axhline(y = threshold, color = 'r', linestyle = '-')
ax[2].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[2].xaxis.set_major_locator(loc)
fig.subplots_adjust(top=0.8)
fig.tight_layout()
plt.show()

print(COEFF_ADT4)

### CASE 5: SINDy with adaptive $\lambda$ parameter and adaptive weights

In [ ]:
# TEMPORAL MODE 1
A1_DT5 = SINDy_MODEL(COEFF_ADT5 [:, 0]).to(processor)

# TEMPORAL MODE 21
A2_DT5 = SINDy_MODEL(COEFF_ADT5 [:, 1]).to(processor)

# TEMPORAL MODE 3
A3_DT5 = SINDy_MODEL(COEFF_ADT5 [:, 2]).to(processor)

Loss_data5    = torch.empty(size=(Epochs, 1))
loss_function = nn.MSELoss()

for epoch in range(Epochs):
    A1_out, A2_out, A3_out  = A1_DT5 (A_candidates), A2_DT5 (A_candidates), A3_DT5 (A_candidates)
    output_data  = torch.stack((A1_out , A2_out, A3_out), dim = 1)
    loss_epoch   = loss_function (A1A2A3_time_deriv, output_data) + torch.abs(Lambda5 + 1e-12)*torch.linalg.matrix_norm(torch.abs(WEIGHTS5)*COEFF_ADT5, ord =1)
    
    optim_COEFF_ADT5.zero_grad()
    optim_Lambda5.zero_grad()
    optim_weights5.zero_grad()
    loss_epoch.backward()

    with torch.no_grad():
        optim_COEFF_ADT5.step()
        optim_Lambda5.step()
        optim_weights5.step()
        Loss_data5 [epoch] = loss_epoch.detach()
        COEFF_ADT5 [torch.abs(COEFF_ADT5) < 0.005] = 0.0
            
    print('LOSS DATA, [EPOCH =', epoch,  ']:',  Loss_data5 [epoch].item())
    print('LEARNING RATE:', optim_COEFF_ADT5.param_groups[0]['lr'])
    print ("*"*85)
       
    scheduler_ADT5.step()
    scheduler_LAMBDA5.step()
    scheduler_weights5.step()
    

In [ ]:
plt.plot(Loss_data5.detach().cpu().numpy())
plt.xlabel('Epoch') 
plt.ylabel('Loss')
plt.yscale("log")
plt.show()

loc = plticker.MultipleLocator(base=3) # this locator puts ticks at regular intervals
fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize =(14, 4))
fig.suptitle('Adaptive $\lambda $ + Adaptive Weights')

ax[0].plot(COEFF_ADT5 [:, 0].detach().cpu().numpy(), 'o') 
ax[0].set_title('X')
ax[0].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[0].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[0].xaxis.set_major_locator(loc)

ax[1].plot(COEFF_ADT5 [:, 1].detach().cpu().numpy(), 'o') 
ax[1].set_title('Y')
ax[1].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[1].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[1].xaxis.set_major_locator(loc)

ax[2].plot(COEFF_ADT5 [:, 2].detach().cpu().numpy(), 'o') 
ax[2].set_title('Z')
ax[2].axhline(y = threshold, color = 'r', linestyle = '-')
ax[2].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[2].xaxis.set_major_locator(loc)
fig.subplots_adjust(top=0.8)
fig.tight_layout()
plt.show()

print(COEFF_ADT5)

### CASE 6: SINDy with adaptive weights

In [ ]:
# TEMPORAL MODE 1
A1_DT6 = SINDy_MODEL(COEFF_ADT6 [:, 0]).to(processor)

# TEMPORAL MODE 2
A2_DT6 = SINDy_MODEL(COEFF_ADT6 [:, 1]).to(processor)

# TEMPORAL MODE 3
A3_DT6 = SINDy_MODEL(COEFF_ADT6 [:, 2]).to(processor)

Loss_data6    = torch.empty(size=(Epochs, 1))
loss_function = nn.MSELoss()

for epoch in range(Epochs):
    A1_out, A2_out, A3_out  = A1_DT6 (A_candidates), A2_DT6 (A_candidates), A3_DT6 (A_candidates)
    output_data  = torch.stack((A1_out , A2_out, A3_out), dim = 1)
    loss_epoch   = loss_function (A1A2A3_time_deriv, output_data) + torch.linalg.matrix_norm(torch.abs(WEIGHTS6)*COEFF_ADT6, ord =1)
    
    optim_COEFF_ADT6.zero_grad()
    optim_weights6.zero_grad()
    loss_epoch.backward()

    with torch.no_grad():
        optim_COEFF_ADT6.step()
        optim_weights6.step()
        Loss_data6 [epoch] = loss_epoch.detach()
        COEFF_ADT6 [torch.abs(COEFF_ADT6) < 0.005] = 0.0
            
    print('LOSS DATA, [EPOCH =', epoch,  ']:',  Loss_data6 [epoch].item())
    print('LEARNING RATE:', optim_COEFF_ADT6.param_groups[0]['lr'])
    print ("*"*85)
       
    scheduler_ADT6.step()
    scheduler_weights6.step()
    

In [ ]:
plt.plot(Loss_data6.detach().cpu().numpy())
plt.xlabel('Epoch') 
plt.ylabel('Loss')
plt.yscale("log")
plt.show()

loc = plticker.MultipleLocator(base=3) # this locator puts ticks at regular intervals
fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize =(14, 4))
fig.suptitle('Adaptive Weights')

ax[0].plot(COEFF_ADT6 [:, 0].detach().cpu().numpy(), 'o') 
ax[0].set_title('X')
ax[0].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[0].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[0].xaxis.set_major_locator(loc)

ax[1].plot(COEFF_ADT6 [:, 1].detach().cpu().numpy(), 'o') 
ax[1].set_title('Y')
ax[1].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[1].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[1].xaxis.set_major_locator(loc)

ax[2].plot(COEFF_ADT6 [:, 2].detach().cpu().numpy(), 'o') 
ax[2].set_title('Z')
ax[2].axhline(y = threshold, color = 'r', linestyle = '-')
ax[2].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[2].xaxis.set_major_locator(loc)
fig.subplots_adjust(top=0.8)
fig.tight_layout()
plt.show()

print(COEFF_ADT6)

In [ ]:
threshold = 0.0005
#****************************************************************************#
loc = plticker.MultipleLocator(base=3) # this locator puts ticks at regular intervals
fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize =(14, 4))
fig.suptitle('$\lambda_{fixed} = 0.001$')

ax[0].plot(COEFF_ADT1 [:, 0].detach().cpu().numpy(), 'o') 
ax[0].set_title('X')
ax[0].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[0].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[0].xaxis.set_major_locator(loc)

ax[1].plot(COEFF_ADT1 [:, 1].detach().cpu().numpy(), 'o') 
ax[1].set_title('Y')
ax[1].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[1].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[1].xaxis.set_major_locator(loc)

ax[2].plot(COEFF_ADT1 [:, 2].detach().cpu().numpy(), 'o') 
ax[2].set_title('Z')
ax[2].axhline(y = threshold, color = 'r', linestyle = '-')
ax[2].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[2].xaxis.set_major_locator(loc)
fig.subplots_adjust(top=0.8)
fig.tight_layout()
plt.show()

#****************************************************************************#
loc = plticker.MultipleLocator(base=3) # this locator puts ticks at regular intervals
fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize =(14, 4))
fig.suptitle('$\lambda_{fixed} = 0.0001$')

ax[0].plot(COEFF_ADT2 [:, 0].detach().cpu().numpy(), 'o') 
ax[0].set_title('X')
ax[0].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[0].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[0].xaxis.set_major_locator(loc)

ax[1].plot(COEFF_ADT2 [:, 1].detach().cpu().numpy(), 'o') 
ax[1].set_title('Y')
ax[1].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[1].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[1].xaxis.set_major_locator(loc)

ax[2].plot(COEFF_ADT2 [:, 2].detach().cpu().numpy(), 'o') 
ax[2].set_title('Z')
ax[2].axhline(y = threshold, color = 'r', linestyle = '-')
ax[2].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[2].xaxis.set_major_locator(loc)
fig.subplots_adjust(top=0.8)
fig.tight_layout()
plt.show()


#****************************************************************************#
loc = plticker.MultipleLocator(base=3) # this locator puts ticks at regular intervals
fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize =(14, 4))
fig.suptitle('$\lambda_{fixed} = 0.001$ + Adaptive Weights')

ax[0].plot(COEFF_ADT3 [:, 0].detach().cpu().numpy(), 'o') 
ax[0].set_title('X')
ax[0].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[0].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[0].xaxis.set_major_locator(loc)

ax[1].plot(COEFF_ADT3 [:, 1].detach().cpu().numpy(), 'o') 
ax[1].set_title('Y')
ax[1].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[1].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[1].xaxis.set_major_locator(loc)

ax[2].plot(COEFF_ADT3 [:, 2].detach().cpu().numpy(), 'o') 
ax[2].set_title('Z')
ax[2].axhline(y = threshold, color = 'r', linestyle = '-')
ax[2].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[2].xaxis.set_major_locator(loc)
fig.subplots_adjust(top=0.8)
fig.tight_layout()
plt.show()


#****************************************************************************#
loc = plticker.MultipleLocator(base=3) # this locator puts ticks at regular intervals
fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize =(14, 4))
fig.suptitle('Adaptive $\lambda $')

ax[0].plot(COEFF_ADT4 [:, 0].detach().cpu().numpy(), 'o') 
ax[0].set_title('X')
ax[0].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[0].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[0].xaxis.set_major_locator(loc)

ax[1].plot(COEFF_ADT4 [:, 1].detach().cpu().numpy(), 'o') 
ax[1].set_title('Y')
ax[1].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[1].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[1].xaxis.set_major_locator(loc)

ax[2].plot(COEFF_ADT4 [:, 2].detach().cpu().numpy(), 'o') 
ax[2].set_title('Z')
ax[2].axhline(y = threshold, color = 'r', linestyle = '-')
ax[2].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[2].xaxis.set_major_locator(loc)
fig.subplots_adjust(top=0.8)
fig.tight_layout()
plt.show()

#****************************************************************************#
loc = plticker.MultipleLocator(base=3) # this locator puts ticks at regular intervals
fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize =(14, 4))
fig.suptitle('Adaptive $\lambda $ + Adaptive Weights')

ax[0].plot(COEFF_ADT5 [:, 0].detach().cpu().numpy(), 'o') 
ax[0].set_title('X')
ax[0].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[0].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[0].xaxis.set_major_locator(loc)

ax[1].plot(COEFF_ADT5 [:, 1].detach().cpu().numpy(), 'o') 
ax[1].set_title('Y')
ax[1].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[1].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[1].xaxis.set_major_locator(loc)

ax[2].plot(COEFF_ADT5 [:, 2].detach().cpu().numpy(), 'o') 
ax[2].set_title('Z')
ax[2].axhline(y = threshold, color = 'r', linestyle = '-')
ax[2].axhline(y = -threshold, color = 'r', linestyle = '-')
ax[2].xaxis.set_major_locator(loc)
fig.subplots_adjust(top=0.8)
fig.tight_layout()
plt.show()


The optimised coefficient matrix is further reduced by assuming the terms to be zero within the chosen threshold (0.001). 

In [ ]:
# TRUNCATED SINDy COEFFICIENTS
C1 = torch.clone(COEFF_ADT1[:, 0]).detach()
C1 [torch.logical_and(C1 >= -threshold, C1 <=threshold)] = 0.0

C2 = torch.clone(COEFF_ADT1[:, 1]).detach()
C2 [torch.logical_and(C2 >= -threshold, C2 <=threshold)] = 0.0

C3 = torch.clone(COEFF_ADT1[:, 2]).detach()
C3 [torch.logical_and(C3 >= -threshold, C3 <=threshold)] = 0.0

In [ ]:
A1_DT_Predict = A1_DT2 (A_candidates)
A2_DT_Predict = A2_DT2 (A_candidates)
A3_DT_Predict = A3_DT2 (A_candidates)

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize =(15, 4))
fig.suptitle('Comparison')

ax[0].plot(A1A2A3_time_deriv[0, :].detach().cpu().numpy(), '-o', label = "POD Mode") 
ax[0].plot(A1_DT_Predict.detach().cpu().numpy(), label = "SINDy") 
ax[0].set_title('Mode 1')


ax[1].plot(A1A2A3_time_deriv[1, :].detach().cpu().numpy(), '-o', label = "POD Mode") 
ax[1].plot(A2_DT_Predict.detach().cpu().numpy(), label = "SINDy - $\lambda_{matrix}$") 
ax[1].set_title('Mode 2')


ax[2].plot(A1A2A3_time_deriv[2, :].detach().cpu().numpy(), '-o', label = "POD Mode") 
ax[2].plot(A3_DT_Predict.detach().cpu().numpy(), label = "Adaptive SINDy + $\lambda_{matrix}$")
ax[2].legend()
ax[2].set_title('Mode 3')

fig.subplots_adjust(top=0.8)
plt.show()